## Semi-structured RAG

![](./img/semi.jpg)

출처: https://medium.com/p/d0fc83c7cc37

In [1]:
! pip install -U langchain openai chromadb langchain-experimental

In [2]:
! pip install "unstructured[all-docs]" pillow pydantic lxml pillow matplotlib chromadb tiktoken
# ! brew install poppler
# ! brew install tesseract

In [3]:
from typing import Any

from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

import uuid

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough

In [4]:
import os

path="./data/"

raw_pdf_elements = partition_pdf(
    filename=os.path.join(path, "gs.pdf"),
    extract_images_in_pdf=False,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=path,
)

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
set([str(type(x)) for x in raw_pdf_elements])

{"<class 'unstructured.documents.elements.CompositeElement'>",
 "<class 'unstructured.documents.elements.Table'>"}

In [6]:
category_counts = {}

for element in raw_pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

unique_categories = set(category_counts.keys())
print(category_counts)
print(unique_categories)

{"<class 'unstructured.documents.elements.CompositeElement'>": 8, "<class 'unstructured.documents.elements.Table'>": 7}
{"<class 'unstructured.documents.elements.CompositeElement'>", "<class 'unstructured.documents.elements.Table'>"}


In [7]:
class Element(BaseModel):
    type: str
    text: Any


categorized_elements = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(Element(type="table", text=str(element)))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        categorized_elements.append(Element(type="text", text=str(element)))

table_elements = [e for e in categorized_elements if e.type == "table"]
print(len(table_elements))

text_elements = [e for e in categorized_elements if e.type == "text"]
print(len(text_elements))

7
8


In [8]:
prompt_text = """You are an assistant tasked with summarizing tables and text in Korean. \
Give a concise summary of the table or text. Table or text chunk: {element} """
prompt = ChatPromptTemplate.from_template(prompt_text)
model = ChatOpenAI(temperature=0, model="gpt-4o")
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

In [9]:
tables = [i.text for i in table_elements]
table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})
texts = [i.text for i in text_elements]
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})

In [10]:
id_key = "doc_id"

# texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_summaries)
]

# tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [
    Document(page_content=s, metadata={id_key: table_ids[i]})
    for i, s in enumerate(table_summaries)
]

print(summary_texts[0])
print(summary_tables[0])

page_content='2023년 8월 25일 개정된 복리후생 규정의 목차는 다음과 같습니다:\n\n1. 총칙: 규정의 목적과 적용 범위 설명\n2. 건강검진: 검진 기준 등\n3. 의료비 지원\n4. 경조금 및 경조 휴가\n5. 학자금 지원\n6. 장기근속포상\n7. 카페테리아\n8. 상해보험\n9. 개인퇴직연금(IRP) 지원\n10. 주택 자금 대출\n\n이 규정은 ㈜GS 소속 전 임직원에게 적용되며, 임원 및 사원의 복리증진을 목적으로 합니다.' metadata={'doc_id': '833c0e38-6938-43c3-ba41-9de10f2f019e'}
page_content='이 표는 건강 검진 대상자와 실시 시기를 요약한 것입니다. 본인은 1년에 한 번, 임원 배우자는 1년에 한 번 종합검진을 받습니다. 만 35세 이상의 직원은 1년에 한 번, 그 배우자는 2년에 한 번 종합검진을 받습니다. 만 34세 이하의 임직원은 1년에 한 번 일반검진을 받습니다.' metadata={'doc_id': '7e84ff00-dd11-4f33-b8a2-e4dd45545a05'}


In [11]:
# MultiVectorRetriever

# 상위 문서를 저장합니다: 상위 문서는 바이트 저장소(일반적으로 InMemoryStore의 인스턴스)에 저장됩니다.
# 이 저장소 계층은 여러 개의 하위 문서를 포함할 수 있는 전체 상위 문서를 유지합니다.
store = InMemoryStore()

# 하위 문서를 저장합니다: 그런 다음 하위 문서는 OpenAIEmbeddings()와 같은 임베딩 함수를 사용하여 벡터 임베딩으로 변환됩니다. 
# 벡터 임베딩은 이러한 임베딩을 관리하기 위한 벡터 저장소 역할을 하는 Chroma 클래스의 인스턴스에 저장됩니다.
vectorstore = Chroma(collection_name="summaries", embedding_function=OpenAIEmbeddings())

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))

In [12]:
template = """Answer the question based only on the following context in Korean, which can include text and tables: {context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(temperature=0, model="gpt-4o")

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [13]:
chain.invoke("본인 결혼 휴가는 몇일이야?")

'본인 결혼 휴가는 5일이야.'

In [14]:
chain.invoke("의료비 지원에서 치과 보철의 적용율은?")

'의료비 지원에서 치과 보철의 적용율은 50%입니다.'

In [15]:
chain.invoke("의료비 지원에서 성형 적용율은?")

'의료비 지원에서 성형은 지원 제외 항목에 해당하므로 적용율은 0%입니다.'